In [1]:
from newspaper import Article
import pandas as pd
import nltk
import io
import numpy as np
import re
import warnings
from nltk.corpus import stopwords
from collections import  Counter
from textblob import TextBlob
import spacy
df = pd.read_excel('input.xlsx')
def get_text(url):
    url1 = url
    article = Article(url1, language="en")
    try:
        article.download()
        article.parse()
        article.nlp()
        return article.title+article.text
    except:
        return ""
for i in range(len(df)):
    var = str(df.iloc[i,0])
    fname = var+'.txt'
    file = io.open(fname,'a',encoding = "utf-8")
    df['URL_ID'][i] = get_text(df['URL'][i])
    file.write(get_text(df['URL'][i]))
    file.close()
df.rename({'URL_ID':'Text'},axis=1,inplace=True)
def transform(text):
    review = re.sub('[^a-zA-Z0-9]', ' ',text) 
    review = review.lower()                   
    review = review.split()
    review = [word for word in review if not word in stopwords.words('english')]   # remove stopwords.
    review = ' '.join(review)
    return review
df['Transform_Text'] = df['Text'].apply(transform)
df['word_counts'] = df['Transform_Text'].apply(lambda x: len(str(x).split()))    
df['average number of words per sentence'] = np.nan

for i in range(0,len(df)):
    df['average number of words per sentence'][i] = df['word_counts'][i]/len(nltk.sent_tokenize(df['Text'][i]))
def char_count(x):
    s = x.split()
    x = ''.join(s)
    return len(x)      
df['chara_count'] = df['Transform_Text'].apply(lambda x: char_count(x))
df['average word length'] = np.nan
for i in range(0,len(df)):
    df['average word length'][i] = df['chara_count'][i]/df['word_counts'][i]
def syllable_count(x):
    v = []
    d = {}
    for i in x:
        if i in "aeiou":
            v.append(i)
            d[i] = d.get(i,0)+1  
    k = []
    for i in d:
        k.append(d[i])
    return np.sum(k)
df['syllable count'] = df['Transform_Text'].apply(lambda x: syllable_count(x))
def complex_word_count(x):
    syllable = 'aeiou'
    t = x.split()
    v = []
    for i in t:
        words = i.split()
        c=Counter()
        for word in words:
            c.update(set(word))
        n = 0
        for a in c.most_common():
            if a[0] in syllable:
                if a[1] >= 2:
                    n += 1
        m = 0
        p = []
        for a in c.most_common():
            if a[0] in syllable:
                p.append(a[0])
        if len(p) >= 2:
            m += 1
        if n >= 1 or m >= 1:
            v.append(i)
    return len(v) 
df['complex_count'] = np.nan
df['complex_count'] = df['Transform_Text'].apply(lambda x: complex_word_count(x))
df['sentence length'] = np.nan
df['Average Sentence Length'] = np.nan
df['Percentage of Complex words'] = np.nan
df['Fog Index'] = np.nan
for i in range(0,len(df)):
    df['sentence length'][i]  =   len(nltk.sent_tokenize(df['Text'][i]))
    df['Average Sentence Length'][i] = df['word_counts'][i]/df['sentence length'][i]
    df['Percentage of Complex words'][i] = df['complex_count'][i]/df['word_counts'][i] 
    df['Fog Index'][i] = 0.4 * (df['Average Sentence Length'][i] + df['Percentage of Complex words'][i])
sentiment = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
dfs = sentiment[['Word','Negative','Positive']]
dfs = dfs.dropna()
dfs.isnull().sum()
dfs['word_lower'] = np.nan
warnings.filterwarnings('ignore')
for i in range(50742,len(dfs)):
        dfs['word_lower'][i] = dfs['Word'][i].lower()
def positive(x):
    s = x.split()
    positive = 0
    for i in dfs['word_lower']:
        if i in s:
            if dfs[dfs['word_lower']==i].Positive.any() == True:
                positive += 1
    return positive
df['positive_score'] = np.nan
for i in range(len(df)):
    df['positive_score'][i] = positive(df['Transform_Text'][i])
def negative_score(x):
    s = x.split()
    negative = 0
    for i in dfs['word_lower']:
        if i in s:
            if dfs[dfs['word_lower']==i].Negative.any() == True:
                negative += 1
    return negative
df['negative_score'] = np.nan
for i in range(len(df)):
    df['negative_score'][i] = negative_score(df['Transform_Text'][i])
df['Polarity Score'] = np.nan
for i in range(len(df)):
    df['Polarity Score'][i] = (df['positive_score'][i]-df['negative_score'][i])/ ((df['positive_score'][i] + df['negative_score'][i]) + 0.000001)
blob = TextBlob(df['Transform_Text'][1])
blob.sentiment
TextBlob(df['Transform_Text'][1]).sentiment[1]
df['subjectivity'] = np.nan
for i in range(len(df)):
    df['subjectivity'][i] = TextBlob(df['Transform_Text'][i]).sentiment[1]
nlp = spacy.load('en_core_web_sm')
df['PERSONAL PRONOUNS'] = np.nan
for i in range(len(df)):
    doc = nlp(df['Text'][i])
    tok = []
    for token in doc:
        if token.pos_ == 'PRON':
            tok.append(token)  
    df['PERSONAL PRONOUNS'][i] = tok
submit = df[['URL','positive_score','negative_score','Polarity Score','subjectivity','Average Sentence Length','Percentage of Complex words',
            'Fog Index','average number of words per sentence','complex_count','word_counts','syllable count','PERSONAL PRONOUNS','average word length']]
file_name = "Output Data Structure.xlsx"
submit.to_excel(file_name)

C:\Users\MUTYALA. SOWMYA\AppData\Local\Temp\ipykernel_11072\4141799258.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['URL_ID'][i] = get_text(df['URL'][i])
C:\Users\MUTYALA. SOWMYA\AppData\Local\Temp\ipykernel_11072\4141799258.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['average number of words per sentence'][i] = df['word_counts'][i]/len(nltk.sent_tokenize(df['Text'][i]))
C:\Users\MUTYALA. SOWMYA\AppData\Local\Temp\ipykernel_11072\4141799258.py:43: RuntimeWarning: invalid value encountered in longlong_scalars
  df['average number of words per sentence'][i] = df['word_counts'][i]/len(nltk.sent_toke